In [5]:
import pandas as pd

In [6]:
content_dir = "//Users/fernando/Documents/Research/LLM_SR_medicine/llm_results/PNP/"
df = pd.read_excel(f"{content_dir}original_data/CompleteSet_20200117_complete_FOrPublication_v2.xlsx", sheet_name="Sheet1")
df['screening1'] = df['Include?'].str.contains('Included', case=False, na=False)
df['screening2'] = df['screening2'].str.contains('Included', case=False, na=False)

# Filtering out rows where 'Duplicate? - Potential dups HL red' contains "not english" in any case and 'Abstract' is empty.
df = df[~df['Duplicate? - Potential dups HL red'].str.contains("not english", case=False, na=False) & df['Abstract'].notna() & df['Abstract'].ne('')]
# Concatenate 'Title' and 'Abstract' with a space in between
df['Record'] = df['Title'].fillna('') + " " + df['Abstract'].fillna('')

# Define the unwanted characters
unwanted_chars = "[\"\\t:\\n\\r\\\\'“”‘’#@\$%\^&\*\(\)-_\+=\{\}\[\]|:;<>,\.\?/~!]"

# Use the replace function to remove unwanted characters
# df['Record'] = df['Record'].str.replace(unwanted_chars, '', regex=True)
df.iloc[0]['Record']

/Users/fernando/opt/anaconda3/envs/chatdrex/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


'Unilateral oculomotor palsy in Charcot-Marie-Tooth disease 1A (CMT 1A) BACKGROUND: Charcot-Marie-Tooth disease (CMT) type 1A is the most common form of CMT 1 and one of the autosomal dominant demyelinating hereditary motor and sensory neuropathies (HMSN). Cranial nerves may be frequently subclinically affected in CMT disease. However manifest clinical signs of cranial nerve involvement are rare. METHODS: This case comprise neurological, ophthalmological, internal medicine and ear-nose-throat investigation, motor and sensory nerve conduction velocity, auditory evoked potentials and orbicularis-oculi reflex measurements, lumbar puncture and blood examination, inclusive molecular genetic testing, as well as electrocardiogram and cranial imaging such as computer tomography and magnetic resonance imaging RESULTS: The present case shows a Charcot-Marie-Tooth (CMT) 1A patient with complete unilateral oculomotor palsy in combination with predominant ipsilateral subclinical trigeminal demyelin

In [7]:
df.to_pickle(f"{content_dir}PNP_preprocessed.pkl")
print(df[['screening1', 'screening2']].value_counts())
print(df.screening1.value_counts())
print(df.screening2.value_counts())
# Counting NaNs per column
nan_count = df.isna().sum()

screening1  screening2
False       False         1539
True        False           94
            True            22
Name: count, dtype: int64
screening1
False    1539
True      116
Name: count, dtype: int64
screening2
False    1633
True       22
Name: count, dtype: int64


In [8]:

checkpoints_dict = {
    "Disease": "If the study investigates a condition that is classified as a form of hereditary peripheral neuropathy and is recognized based on its genetic basis in medical literature, return True. Otherwise, return False.",
    "Treatment": "If the study focuses on pharmacological therapies or genotype-related dietary changes specifically targeting the neuropathy, return True. If the study primarily investigates non-pharmacological interventions such as physiotherapy, surgery, and genetic counseling, return False.",
    "Human": "If the study involves human participants, return True. If it involves only animal models or in vitro studies, return False.",
    "Genetic": "If human participants in the study are confirmed to have hereditary peripheral neuropathy through direct genetic testing or through family genetic association where further individual testing might not be necessary, return True. This includes confirmation through nucleotide-based assays and clinically accepted diagnostic methods such as specific enzyme activity in biopsies, judged on a case-by-case basis to be as definitive as genetic tests. If the study includes participants diagnosed only based on clinical symptoms without genetic confirmation, return False.",
    "Results": "If results are reported specifically for the genetically confirmed cohort in studies including both genetically confirmed and non-confirmed participants, return True. If the study does not segregate data between genetically confirmed and non-confirmed participants, or if results from credible sources and databases are not exclusively from interventional studies with available results, return False."
}
